In [ ]:
import numpy as np

def unit_step(v):
    return 1 if v >= 0 else 0

def perceptron_model(x, weights, bias):
    weighted_sum = np.dot(weights, x) + bias
    return unit_step(weighted_sum)

def train_perceptron(X, y, weights, bias, learning_rate=0.01, max_epochs=1000, tolerance=1e-4, patience=10, verbose=False):
    prev_weights = np.copy(weights)
    patience_counter = 0

    for epoch in range(max_epochs):
        total_error = 0
        for i in range(len(X)):
            y_pred = perceptron_model(X[i], weights, bias)
            error = y[i] - y_pred
            total_error += abs(error)

            # Update weights and bias
            weights += learning_rate * error * X[i]
            bias += learning_rate * error

        # Check for small weight change
        weight_change = np.sum(np.abs(weights - prev_weights))
        if weight_change < tolerance:
            if verbose:
                print(f"Training stopped due to small weight change in epoch {epoch}")
            break

        prev_weights = np.copy(weights)

        # Early stopping based on zero total error
        if total_error == 0:
            if verbose:
                print(f"Training stopped due to zero classification error in epoch {epoch}")
            break

        # Patience mechanism
        if total_error > 0:
            patience_counter += 1
            if patience_counter >= patience:
                if verbose:
                    print(f"Training stopped due to lack of improvement in epoch {epoch}")
                break
        else:
            patience_counter = 0  # Reset counter if there was improvement

    return weights, bias

def logic_gate(x, weights, bias):
    return perceptron_model(x, weights, bias)

def XOR_logic_function(x):
    y1 = logic_gate(x, np.array([1, 1]), -1.5)
    y2 = logic_gate(x, np.array([1, 1]), -0.5) 
    y3 = logic_gate(np.array([y1]), np.array([-1]), 0.5) 

    return logic_gate(np.array([y2, y3]), np.array([1, 1]), -1.5)  # Final AND gate

# Test the XOR function with all possible inputs
X = np.array([[0, 0], [0, 1], [1, 0], [1, 1]])
Y = np.array([0, 1, 1, 0])

for x in X:
    print(f"XOR({x[0]}, {x[1]}) = {XOR_logic_function(x)}")


XOR(0, 0)=0
XOR(0, 1)=1
XOR(1, 0)=1
XOR(1, 1)=0
